In [16]:
pip install ale_py

  Using cached ale_py-0.8.1-cp39-cp39-win_amd64.whl (952 kB)
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [21]:
import gym
import ale_py

print('gym:', gym.__version__)
print('ale_py:', ale_py.__version__)


gym: 0.26.2
ale_py: 0.8.1


In [3]:
pip install gym[all]

Note: you may need to restart the kernel to use updated packages.
  Using cached imageio-2.27.0-py3-none-any.whl (3.4 MB)
  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached lz4-4.3.2-cp39-cp39-win_amd64.whl (99 kB)
  Using cached pytest-7.0.1-py3-none-any.whl (296 kB)
  Using cached ale_py-0.8.1-cp39-cp39-win_amd64.whl (952 kB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached mujoco_py-2.1.2.14-py3-none-any.whl (2.4 MB)
  Using cached mujoco-2.2.0-cp39-cp39-win_amd64.whl (2.8 MB)
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Using cached pluggy-1.0.0-py2.py3-none-any.whl (13 kB)
  Using cached iniconfig-2.0.0-py3-none-any.whl (5.9 kB)
  Using cached py-1.11.0-py2.py3-none-any.whl (98 kB)
  Using cached atomicwrites-1.4.1-py2.py3-none

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      Using setuptools (version 65.6.3).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\Box2D
      copying library\Box2D\Box2D.py -> build\lib.win-amd64-cpython-39\Box2D
      copying library\Box2D\__init__.py -> build\lib.win-amd64-cpython-39\Box2D
      creating build\lib.win-amd64-cpython-39\Box2D\b2
      copying library\Box2D\b2\__init__.py -> build\lib.win-amd64-cpython-39\Box2D\b2
      running build_ext
      building 'Box2D._Box2D' extension
      swigging Box2D\Box2D.i to Box2D\Box2D_wrap.cpp
      swig.exe -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library\Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D\Box2D_wrap.cpp Box2D\Box2D.i
      Box2D\Common\b2Math

In [22]:
env = gym.make("MsPacman-v0")

NameNotFound: Environment MsPacman doesn't exist. 

In [ ]:
pip install gym[accept-rom-license]

In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [7]:
import random
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from collections import deque

In [9]:
env = gym.make("MsPacman-v4", render_mode="human")
state_size = (88, 80, 1)
action_space = env.action_space.n

NameNotFound: Environment MsPacman doesn't exist. 

In [10]:
color = np.array([210, 164, 74]).mean()


def preprocess_state(state):
    image = state[1:176:2, ::2]
    image = image.mean(axis=2)
    image[image == color] = 0
    image = (image - 128) / 128 - 1
    image = np.expand_dims(image.reshape(88, 80, 1), axis=0)
    return image

In [11]:
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.replay_buffer = deque(maxlen=5000)
        self.gamma = 0.9
        self.epsilon = 0.8
        self.update_rate = 1000
        self.main_network = self.build_network()
        self.target_network = self.build_network()
        self.target_network.set_weights(self.main_network.get_weights())

    def build_network(self):
        model = Sequential()
        model.add(
            Conv2D(32, (8, 8), strides=4, padding="same", input_shape=self.state_size)
        )
        model.add(Activation("relu"))

        model.add(Conv2D(64, (4, 4), strides=2, padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(64, (3, 3), strides=1, padding="same"))
        model.add(Activation("relu"))

        model.add(Flatten())
        model.add(Dense(512, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam())
        return model

    def store_transition(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def epsilon_greedy(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return np.random.randint(self.action_size)

        Q_values = self.main_network.predict(state)
        return np.argmax(Q_values[0])

    def train(self, batch_size):
        minibatch = random.sample(self.replay_buffer, batch_size)
        for state, action, reward, next_state, done in minibatch:
            if not done:
                target_Q = reward + self.gamma * np.amax(
                    self.target_network.predict(next_state)
                )
            else:
                target_Q = reward

        Q_values = self.main_network.predict(state)
        Q_values[0][action] = target_Q
        self.main_network.fit(state, Q_values, epochs=1, verbose=0)

    def update_target_network(self):
        self.target_network.set_weights(self.main_network.get_weights())

In [ ]:
env = gym.make("MsPacman-v4", render_mode="rgb_array")
state_size = (88, 80, 1)
action_size = env.action_space.n
print(action_size)
print(state_size)
num_episodes = 1
num_timesteps = 10
batch_size = 8
num_screens = 4
dqn = DQN(state_size, action_size)
done = False
time_step = 0
print()
for i in range(num_episodes):
    Return = 0
    s, info = env.reset()

    state = preprocess_state(s)
    for t in range(num_timesteps):
        env.render()
        time_step += 1
        if time_step % dqn.update_rate == 0:
            dqn.update_target_network()

        action = dqn.epsilon_greedy(state)
        next_state, reward, done, info, _ = env.step(action)
        next_state = preprocess_state(next_state)
        dqn.store_transition(state, action, reward, next_state, done)
        state = next_state
        Return += reward
        if done:
            print("Episode: ", i, "....", "Return", Return)
            break

        if len(dqn.replay_buffer) > batch_size:
            dqn.train(batch_size)

In [ ]:
import gym

env = gym.make("MsPacman-v4", render_mode="human")
import time

num_steps = 1500
obs = env.reset()
for _ in range(num_steps):
    action = env.action_space.sample()
    obs, reward, done, info, k = env.step(action)
    env.render()
    time.sleep(0.001)
    if done:
        env.reset()

# Close the env
env.close()